In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10280632972717285
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 12.444460539354026
200000 22.52482282919519
300000 24.83485375962433
400000 27.90510738595811
500000 28.639280952402054
600000 29.46062968863721
700000 28.480504349383803
800000 27.898545614854093
900000 27.726208040426112
1000000 27.873034346970805
1100000 28.213501778002502
1200000 13.461480425204225
1300000 23.869086595280745
1400000 27.59552145402857
1500000 27.843142140570297
1600000 27.691069538672416
1700000 27.960202634194953
1800000 29.391208065088016
1900000 28.06145416627415
2000000 27.759826282301926
2100000 28.358287798755413
2200000 28.654090696000427
2300000 29.108751245372595
2400000 7.502183693043104
2500000 30.06691358268006
2600000 35.429828885194006
2700000 35.00732034359719
2800000 35.00304467339041
2900000 35.191265366431
3000000 33.7

29400000 18.955874668342336
29500000 16.247904638818717
29600000 25.971064011050057
29700000 26.630067288084366
29800000 26.861095591540522
29900000 28.252632935450944
30000000 26.795263076178156
30100000 28.064719427241947
30200000 28.421238443619757
30300000 28.21937080735676
30400000 12.282063496148531
30500000 29.01878148065038
30600000 28.501150265925713
30700000 29.566098071195483
30800000 25.4622687235026
30900000 14.89679152403717
31000000 28.42200027806735
31100000 29.308263231997852
31200000 28.53160832914235
31300000 28.058282882466266
31400000 27.803304052168205
31500000 27.93658287540764
31600000 27.4693678725782
31700000 26.549842426647693
31800000 7.694074696021782
31900000 27.398529387839773
32000000 26.3318534230037
32100000 27.791920189539944
32200000 28.064279294675238
32300000 28.239209527300726
32400000 27.998858981619957
32500000 29.362789610243826
32600000 28.62143975673211
32700000 19.192561823596563
32800000 21.608758637103854
32900000 29.019229711506398
330000

59100000 26.703381182839852
59200000 26.50333144584124
59300000 11.340969839047215
59400000 24.504801793650753
59500000 24.761897313945546
59600000 25.141795461799266
59700000 23.333520854311065
59800000 23.622378274287257
59900000 23.99948099806676
60000000 13.516023741531065
60100000 17.633902726342512
60200000 24.355327872196604
60300000 23.862853039140063
60400000 23.416909645133632
60500000 25.03208178581197
60600000 24.574579571111993
60700000 21.029877469419407
60800000 16.562201739755324
60900000 26.564084130418944
61000000 26.742533638734802
61100000 26.285306055365506
61200000 26.280540382729747
61300000 25.930040502639656
61400000 26.378407166366568
61500000 26.50811578767621
61600000 25.301717753261347
61700000 10.104018205000376
61800000 24.87649546981998
61900000 24.513242025969223
62000000 23.755726727631004
62100000 23.616272498190046
62200000 22.562847853749982
62300000 23.8461122720242
62400000 5.956194967846685
62500000 24.001243568249336
62600000 22.40668711022682
6

88800000 10.081743197909395
88900000 21.63994866347705
89000000 22.478466305386767
89100000 22.58422501586965
89200000 23.487896335304605
89300000 22.09244415863954
89400000 23.51795179513033
89500000 23.347696434286313
89600000 23.025136640041204
89700000 23.63124302544007
89800000 22.763763820400225
89900000 22.074508353090057
90000000 21.435850554180917
90100000 11.848414598526228
90200000 17.251533618091134
90300000 20.019802708882192
90400000 21.616182096838088
90500000 20.606439851906817
90600000 9.161138932805963
90700000 18.554129083761275
90800000 20.60541779879719
90900000 20.2010882661111
91000000 20.4993399404689
91100000 10.416810424888547
91200000 19.527683984793967
91300000 21.763473707576924
91400000 21.46608525415332
91500000 22.403559066229167
91600000 22.295382817387292
91700000 23.178726255063935
91800000 23.46361329613056
91900000 22.764754718583465
92000000 22.94431267322207
92100000 23.602420801473524
92200000 22.683774438398753
92300000 21.968286484655522
924000

117900000 20.457231210989946
118000000 19.92474455689129
118100000 19.23672981668229
118200000 19.545163550572354
118300000 18.526473111946686
118400000 18.810862786238705
118500000 16.457996140428328
118600000 9.223364968635076
118700000 19.25329321110315
118800000 12.71129144844558
118900000 10.921704648720901
119000000 18.265030310777146
119100000 11.794921911524089
119200000 15.266855314494132
119300000 19.11245775239214
119400000 18.15722073688908
119500000 19.070531134887265
119600000 18.964974338649185
119700000 19.554801049358826
119800000 20.1872460303429
119900000 21.135079844369436
120000000 20.31210789740431
120100000 20.93740931657696
120200000 20.80129303282803
120300000 20.21605698216378
120400000 19.46236861519781
120500000 19.231641963854013
120600000 19.29350259044255
120700000 19.095415171984005
120800000 18.55707106436568
120900000 12.141908036882516
121000000 13.342400764486515
121100000 18.053933469462272
121200000 5.37782036157246
121300000 18.68894960346434
1214

...

261100000 10.278328972872345
261200000 10.307686132915485
261300000 9.969576524383827
261400000 10.08907276438036
261500000 8.578531881531884
261600000 5.788150329187693
261700000 9.974701853806799
261800000 9.569456061825028
261900000 9.56780551515004
262000000 9.551106761053092
262100000 9.263454921660694
262200000 9.043389214041017
262300000 7.306277607823036
262400000 4.815159922190873
262500000 8.860458295497274
262600000 9.2134422364545
262700000 9.424392624154374
262800000 9.497181395755257
262900000 9.658793733688062
263000000 9.98253226655702
263100000 8.043630316592415
263200000 5.72523929501994
263300000 10.032956202577699
263400000 9.932743392114684
263500000 10.091832380919053
263600000 10.101876298642043
263700000 9.904202155700089
263800000 9.989457738117533
263900000 3.828810012363268
264000000 10.591732615978728
264100000 9.695974019770448
264200000 9.34057148727992
264300000 9.218758874038599
264400000 9.237540975315433
264500000 8.796679810278524
264600000 8.85735458

290500000 7.570859096361383
290600000 1.8521028930893946
290700000 7.7681075268225905
290800000 7.735158674789138
290900000 7.845893298650378
291000000 7.787846247205292
291100000 7.6467470372720445
291200000 7.512700449962049
291300000 7.884629879052617
291400000 7.856767527070885
291500000 8.029354424560964
291600000 2.826098318509741
291700000 8.069513008156658
291800000 7.842939633550997
291900000 4.23326879983543
292000000 7.006349608937732
292100000 7.696372193895844
292200000 7.566640987844867
292300000 7.648369898213234
292400000 7.534680310767974
292500000 7.401417013299821
292600000 7.5345584952568165
292700000 7.432934113410256
292800000 7.192914537314763
292900000 4.785866868313749
293000000 4.664795078823675
293100000 7.302019242275329
293200000 7.487548325992457
293300000 7.450939648184134
293400000 7.674055047052861
293500000 7.718579997079915
293600000 7.764793562199496
293700000 7.481607595349625
293800000 7.7383712527389825
293900000 7.2749924699317265
294000000 3.582

319300000 2.579420295766856
319400000 2.5199185419282313
319500000 2.3795658415684513
319600000 2.3513046496228336
319700000 2.3810827492277093
319800000 2.3984970644291033
319900000 2.468267105100876
320000000 2.7262585437520466
320100000 2.524504574921868
320200000 2.6712523117641687
320300000 2.419798084202011
320400000 2.372030947025043
320500000 2.504018648696804
320600000 2.419826057342163
320700000 2.0852860187026225
320800000 2.4195091727615297
320900000 2.2327788225349545
321000000 1.740069049942657
321100000 1.2979060241099438
321200000 1.2427587117312464
321300000 1.2456788774583691
321400000 1.5135024112493753
321500000 2.403659420453839
321600000 2.6750569505146493
321700000 2.209320454528664
321800000 1.9728070113146252
321900000 2.4437660682249853
322000000 2.374587798594427
322100000 2.3791234454756305
322200000 2.373247020303153
322300000 2.6905316638306975
322400000 2.4611898489568613
322500000 2.474254635257613
322600000 2.4527221555264354
322700000 2.413730158776204

347900000 1.3966918630144656
348000000 1.4646605282901097
348100000 1.4341929686393915
348200000 1.4565314433272851
348300000 1.6269708240428071
348400000 1.3595179711714627
348500000 1.386747025620605
348600000 1.3652811245395986
348700000 1.4339910179495494
348800000 1.1380131946379688
348900000 1.2627238550132664
349000000 1.4621436621283372
349100000 1.3925206774025825
349200000 0.9923488565659655
349300000 0.6664158399096967
349400000 0.711945857229617
349500000 0.726754475312198
349600000 0.9142842661711496
349700000 1.101731870624241
349800000 1.4520215256723776
349900000 1.1975774594820698
350000000 1.281857245361995
350100000 1.3030951630855103
350200000 1.2278938913168491
350300000 1.2831830915243825
350400000 1.4046239367497346
350500000 1.3441977842303259
350600000 1.4862685182094397
350700000 1.3677406742157392
350800000 1.3568704918616135
350900000 1.2828069996616398
351000000 1.249816302945837
351100000 1.2764947471393704
351200000 1.2590863695607524
351300000 1.22406133

375700000 0.06417009859658677
375800000 0.06344473626084393
375900000 0.06275609346541829
376000000 0.06172667163938284
376100000 0.06078761549890373
376200000 0.0607145370205409
376300000 0.060255555716979506
376400000 0.05912792218093409
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.30463108439829545
376600000 0.3564157201092223
376700000 0.3615937692267034
376800000 0.3491815181346191
376900000 0.34298471511047085
377000000 0.3200394114510331
377100000 0.33216371100207304
377200000 0.26553516609447597
377300000 0.2568745526284841
377400000 0.25960918974936137
377500000 0.2518489352370461
377600000 0.24808693299196627
377700000 0.24988998416215383
377800000 0.24421058255562186
377900000 0.2432217212484055
378000000 0.23880403560132982
378100000 0.23317010147383216
378200000 0.24705468893311955
378300000 0.2429290283468776
378400000 0.2355251353379309
378500000 0.2673633331779719
378600000 0.27545776016408074
378700000 0.2616900678823014
378800000 0